In [ ]:
import pynapple as nap
from pathlib import Path

# see excel sheet for details
nwb_paths = [
    Path("/data_store2/neuropixels/nwb/NP112_B1/NP112_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP113_B1/NP113_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP114_B1/NP114_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP116_B1/NP116_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP116_B2/NP116_B2.nwb"),
    Path("/data_store2/neuropixels/nwb/NP122_B1/NP122_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP128_B1/NP128_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP129_B1/NP129_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP131_B1/NP131_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP132_B2/NP132_B2.nwb"),
    Path("/data_store2/neuropixels/nwb/NP132_B3/NP132_B3.nwb"),
    Path("/data_store2/neuropixels/nwb/NP136_B1/NP136_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP137_B1/NP137_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP138_B1/NP138_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP139_B1/NP139_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP139_B2/NP139_B2.nwb"),
    Path("/data_store2/neuropixels/nwb/NP145_B1/NP145_B1.nwb"),
    Path("/data_store2/neuropixels/nwb/NP147_B2/NP147_B2.nwb"),
    Path("/data_store2/neuropixels/nwb/NP150_B1/NP150_B1.nwb"),
]    

#    Path("/data_store2/neuropixels/nwb/NP130_B1/NP130_B1.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP130_B2/NP130_B2.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP133_B1/NP133_B1.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP140_B1/NP140_B1.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP140_B2/NP140_B2.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP141_B1/NP141_B1.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP151_B1/NP151_B1.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP151_B2/NP151_B2.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP152_B2/NP152_B2.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP152_B3/NP152_B3.nwb"),
#    Path("/data_store2/neuropixels/nwb/NP153_B1/NP153_B1.nwb"),

In [ ]:
import fnmatch
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from scipy.linalg import norm
from scipy.stats import kstest
from scipy.stats import ttest_rel

# define variables to save out from loop
prod_neuron_indices_list = []
sens_neuron_indices_list = []
prod_neuron_perc_list = []
sens_neuron_perc_list = []
both_neuron_perc_list = []
prod_neuron_ct_list =  []
sens_neuron_ct_list = []
both_neuron_ct_list = []
total_neuron_ct_list = []
tumor_id_list = []
infiltrated_id_list = []
insertion_id_list = []
i_save = 0

# loop through all nwb files
for i in range(len(nwb_paths)):
    data = nap.load_file(nwb_paths[i])
    keys = data.keys()

    # find the spike TsGroup objects, number various based on recording apparatus
    template = "*imec*"
    keys = [key for key in keys if fnmatch.fnmatch(key, template)]
    # Preferentially select keys containing 'KS4'; if none, use all keys
    ks_keys = [key for key in keys if "KS4" in key]
    if ks_keys:
        keys = ks_keys
    # Preferentially select keys containing 'Th='; if none, use all keys
    th8_keys = [key for key in keys if "Th=8" in key]
    if th8_keys:
        keys = th8_keys
    else:
        th_keys = [key for key in keys if "Th=" in key]
        if th_keys:
            keys = th_keys
    template = "*sentgen*" 
    keys = [key for key in keys if not fnmatch.fnmatch(key, template)]

    print(data)
    for s in range(len(keys)):
        spike_times = data[keys[s]]
        # print(spike_times)
        
        # 1. filter to good neurons
        firingRates_all = spike_times.metadata["rate"]
        task_times = data["TaskTimes"]
        beh_epochs = nap.IntervalSet(start=task_times.start, end=task_times.end)
        spike_times_beh = spike_times.restrict(beh_epochs)
        firingRates_beh = spike_times_beh.metadata["rate"]

        # calculate ks stats
        start_time = task_times.start
        end_time = task_times.end
        ks_stats = np.zeros(len(spike_times))
        ks_pvals = np.zeros(len(spike_times))
        for u in range(len(spike_times)):
            # get spike times for this unit
            test = spike_times[u].as_series().index.values

            # The null hypothesis is that the spike times are uniformly distributed between start_time and end_time
            if len(test) > 1:
                min_time = start_time[0]
                max_time = end_time[-1]
                # Normalize spike times to [0, 1] for KS test against uniform
                normalized_spike_times = (test - min_time) / (max_time - min_time)
                ks_result = kstest(normalized_spike_times, 'uniform')
                ks_stats[u] = ks_result.statistic
                ks_pvals[u] = ks_result.pvalue
            else:
                ks_stats[u] = np.nan
                ks_pvals[u] = np.nan

        # now do ISI calculation
        violationThreshold = 3/1000 #for 3 ms isi refractory period, what used before, middle of range that kilosort cites
        violationPct = np.zeros(len(spike_times))
        for u in range(len(spike_times)):
            unit = spike_times[u]
            unit = unit.as_series().index
            if len(unit) < 100:
                violationPct[u] = 1 
            else:
                isi = unit.diff()[1:len(unit)]
                violations = np.where(isi<violationThreshold)
                violations = np.array(violations)
                violationPct[u] = violations.size/len(isi)

        # now apply masks
        if "KSLabel" in spike_times.metadata:
            KSLabels = spike_times.metadata["KSLabel"]
        else:
            KSLabels = spike_times.metadata["quality"]
        firingRates = firingRates_beh
        mask1 = violationPct<3/100 # throwing out neurons with > 3% isi violations with isi = 3 ms; lose quite a bit with 5 ms esp when including cdf stats
        mask2 = firingRates>0.5 # firing rate > 0.5 ms during all behavior epochs
        mask3 = KSLabels!="noise" # not noise per auto label
        mask4 = ks_stats < 0.3 # KS stat < 0.3 (stability across recording)
        mask = mask1 & mask2 & mask3 & mask4
        indicesFinal = firingRates.index[mask]
        spike_times_good = spike_times_beh[indicesFinal]
        print(f"Subject: {i}, insertion: {s}, Initial Number: {len(spike_times)}, Number of good neurons: {len(spike_times_good)}")